In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import sklearn.linear_model as lm
import sklearn.svm as svm
from sklearn.externals import joblib
import os
import sys
from sample_feature import TestFeature
import digits_functions as digits
import findspark
findspark.init(os.path.expanduser('~/model-serving/spark-1.6.0-bin-hadoop2.4'))
# findspark.init()

import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionModel, LogisticRegressionWithSGD
from pyspark.mllib.classification import SVMModel, SVMWithSGD
# from pyspark.mllib.tree import RandomForestModel

[1, 10]


/Users/crankshaw/anaconda2/envs/clipper/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [ ]:
# def load_digits(digits_location, digits_filename = "train.data"):
#     digits_path = digits_location + "/" + digits_filename
#     print "Source file:", digits_path
#     df = pd.read_csv(digits_path, sep=",", header=None)
#     data = df.values
#     print "Number of image files:", len(data)
#     y = data[:,0]
#     X = data[:,1:]
#     return (X, y)

# def load_models():
#     model_path = "sklearn_models"
#     features = []
#     for i in range(10):
#         p = "%s/predict_%d_svm/predict_%d_svm.pkl" % (model_path, i, i)
#         print "loading %s" % p
#         features.append(joblib.load(p))
#     return features

# # This code normalizes x as part of the prediction, so don't normalize the test data
# # ahead of time
# def get_features(features, x):
#     return np.array([f.predict(x.reshape(1,-1))[0] for f in features])


# def modify_data(X_train, X_test, ws, drop):
#     dim = X_test.shape[1]-drop
#     means = np.mean(X_train, axis=0)
#     ind = np.argsort(abs(ws)).tolist()[0][::-1][0:dim]
#     #ind = np.random.permutation(X_test.shape[1])[0:dim]
#     features = np.zeros(X_test.shape)
#     for i in range(X_test.shape[1]):
#         if i in ind:
#             features[:,i] = X_test[:,i]
#         else:
#             features[:,i] = means[i]
#     return features

# def drop_features(X_test, ws, drop):
#     dim = X_test.shape[1]-drop
#     ind = np.argsort(abs(ws)).tolist()[0][::-1][0:dim]
#     #ind = np.random.permutation(X_test.shape[1])[0:dim]
#     features = X_test[:,perm]
#     return features




# def run(fs,tasks,method):
#     dic_err = {}
#     dic_num = {}
#     for i in range(len(tasks)):
#         print 'task %d:' % i
#         print 'task preference: ', tasks[i].pref
#         features_train = []
#         for j in range(tasks[i].X.shape[0]):
#             features_train.append(get_features(fs, tasks[i].X[j]))
#         features_train = np.array(features_train)
        
#         clf = lm.LogisticRegression()
#         clf.fit(features_train, tasks[i].y)
#         if method == 'drop':
#             features_train = drop_features(features_train, clf.coef_, drop)
#         clf_new = lm.LogisticRegression()
#         clf_new.fit(features_train, tasks[i].y)
        
#         #print clf_new.coef_
# #         for drop in range(10):
#         for drop in [0]:

#             print 'drop %d features.' % drop
#             features_test=[]
#             for j in range(tasks[i].test_X.shape[0]):
#                 features_test.append(get_features(fs, tasks[i].test_X[j]))
#             features_test = np.array(features_test)
#             if method == 'drop':
#                 features_test = drop_features(features_test, clf.coef_, drop)
#             else:
#                 features_test = modify_data(features_train,features_test, clf.coef_, drop)

#             pred = clf_new.predict(features_test)
#             err = np.sum(pred != tasks[i].test_y)
#             print 'error: ', err
#             if drop in dic_err and drop in dic_num:
#                 dic_err[drop] += err
#                 dic_num[drop] += len(pred)
#             else:
#                 dic_err[drop] = err
#                 dic_num[drop] = len(pred)
    
#     error_list = []
#     for k in dic_err:
#         error_list.append(dic_err[k]/float(dic_num[k]))
#     #print 'average rate error: %f' % (np.mean(accuracy))
#     return error_list

In [2]:
conf = SparkConf() \
            .setAppName("crankshaw-pyspark") \
            .set("spark.executor.memory", "2g") \
            .set("spark.kryoserializer.buffer.mb", "128") \
            .set("master", "local")
sc = SparkContext(conf=conf, batchSize=10)

In [38]:
def load_digits(digits_location, digits_filename = "train.data"):
    digits_path = digits_location + "/" + digits_filename
    print "Source file:", digits_path
    df = pd.read_csv(digits_path, sep=",", header=None)
    data = df.values
    print "Number of image files:", len(data)
    y = data[:,0]
    X = data[:,1:]
    return (X, y)

def load_sk_models():
    model_path = "sklearn_models"
    features = []
    for i in range(10):
        p = "%s/predict_%d_svm/predict_%d_svm.pkl" % (model_path, i, i)
        print "loading %s" % p
        features.append(joblib.load(p))
    return features

# This code normalizes x as part of the prediction, so don't normalize the test data
# ahead of time
# def get_features(features, x):
#     """
#     WORKS FOR SPARK FEATURES
#     """
#     return np.array([f.predict(x.reshape(1,-1))[0] for f in features])



def get_features(features, x):
    """
    WORKS FOR SPARK FEATURES
    """
    return np.array([float(f.predict(x)) for f in features])


def load_spark_models():
    
    base_path = "spark_models/svm_predict_%d"
    models = []
    for i in range(1,11):
        model = SVMModel.load(sc, base_path % i)
        models.append(model)
    return models

def train_tasks(tasks):
    per_task_err = []
    for i in range(len(tasks)):
#         print 'task %d' % i
        features_train = []
        for j in range(tasks[i].X.shape[0]):
            features_train.append(get_features(fs, tasks[i].X[j]))
        features_train = np.array(features_train)
        
#         clf = lm.LogisticRegression(penalty="l1", solver='liblinear', fit_intercept=False, C=1.0)
        clf = lm.LogisticRegression(penalty="l2", C=0.10, solver='liblinear')

        clf.fit(features_train, tasks[i].y)
        features_test=[]
        for j in range(tasks[i].test_X.shape[0]):
            features_test.append(get_features(fs, tasks[i].test_X[j]))
        features_test = np.array(features_test)
        predictions = clf.predict(features_test)
        err = np.sum(predictions != tasks[i].test_y)
        per_task_err.append(float(err)/float(len(predictions)))
        if i % 10 == 0:
            print np.mean(per_task_err)
    return per_task_err

In [27]:
test_x, test_y = load_digits("/Users/crankshaw/model-serving/data/mnist_data", digits_filename="test.data")
# train_x, train_y = load_digits("/Users/crankshaw/model-serving/data/mnist_data", "train.data")
fs = load_spark_models()
print test_y[10], get_features(fs, test_x[10])
my_tasks = digits.create_mtl_datasets(test_x, test_y, nTasks=100, taskSize=30, testSize=50)

Source file: /Users/crankshaw/model-serving/data/mnist_data/test.data
Number of image files: 10000
1 [ 1.  0.  1.  1.  0.  1.  1.  0.  1.  0.]
making task 0
making task 50


In [39]:
errs = train_tasks(my_tasks)
print errs
print np.mean(errs)

0.2
0.161818181818
0.173333333333
0.176774193548
0.18487804878
0.186666666667
0.182950819672
0.183661971831
0.193827160494
0.190549450549
[0.2, 0.22, 0.02, 0.18, 0.18, 0.04, 0.16, 0.16, 0.06, 0.28, 0.28, 0.22, 0.2, 0.04, 0.02, 0.16, 0.28, 0.16, 0.34, 0.2, 0.24, 0.02, 0.28, 0.16, 0.12, 0.08, 0.28, 0.26, 0.12, 0.2, 0.32, 0.2, 0.22, 0.22, 0.32, 0.32, 0.16, 0.3, 0.24, 0.1, 0.02, 0.18, 0.2, 0.26, 0.08, 0.26, 0.2, 0.28, 0.14, 0.0, 0.34, 0.08, 0.22, 0.2, 0.1, 0.2, 0.2, 0.06, 0.26, 0.14, 0.18, 0.22, 0.1, 0.18, 0.16, 0.3, 0.02, 0.3, 0.22, 0.12, 0.26, 0.42, 0.28, 0.28, 0.24, 0.34, 0.08, 0.36, 0.06, 0.3, 0.3, 0.12, 0.34, 0.22, 0.08, 0.06, 0.14, 0.16, 0.16, 0.22, 0.14, 0.18, 0.08, 0.22, 0.14, 0.04, 0.14, 0.12, 0.26, 0.24]
0.1876


In [ ]:
print np.mean(errs)

In [ ]:
def get_spark_features(features, x):
    return np.array([float(f.predict(x)) for f in features])


def load_spark_models():
    conf = SparkConf() \
            .setAppName("crankshaw-pyspark") \
            .set("spark.executor.memory", "2g") \
            .set("spark.kryoserializer.buffer.mb", "128") \
            .set("master", "local")
    sc = SparkContext(conf=conf, batchSize=10)
    base_path = "spark_models/svm_predict_%d"
    models = []
    for i in range(1,11):
        model = SVMModel.load(sc, base_path % i)
        models.append(model)
    return models